In [ ]:
#hurst exponent - persistence
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hurst import compute_Hc
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    729: "MESSINA",
    727: "FIUMEDINISI",
    738: "SAN PIER NICETO",
    739: "TORREGROTTA",
    707: "RIPOSTO",
    764: "SIRACUSA",
    734: "NOVARA DI SICILIA",
    708: "LINGUAGLOSSA",
    769: "PACHINO",
    706: "CATANIA",
    767: "NOTO",
    713: "PEDARA",
    735: "PATTI",
    758: "ISPICA",
    715: "RANDAZZO",
    766: "LENTINI",
    765: "FRANCOFONTE",
    709: "MALETTO",
    770: "PALAZZOLO ACREIDE",
    712: "PATERNò",
    705: "BRONTE",
    733: "NASO",
    711: "MINEO",
    725: "CESARò VIGNAZZA",
    756: "RAGUSA",
    761: "SCICLI",
    730: "MILITELLO ROSMARINO",
    737: "SAN FRATELLO",
    757: "COMISO",
    716: "CALTAGIRONE",
    710: "MAZZARRONE",
    760: "SANTA CROCE CAMERINA",
    723: "CARONIA BUZZA",
    717: "AIDONE",
    721: "NICOSIA",
    762: "ACATE",
    722: "PIAZZA ARMERINA",
    731: "MISTRETTA",
    736: "PETTINEO",
    699: "MAZZARINO",
    752: "GANGI",
    718: "ENNA",
    743: "CASTELBUONO",
    701: "RIESI",
    695: "CALTANISSETTA",
    753: "PETRALIA SOTTANA",
    754: "POLIZZI GENEROSA",
    696: "DELIA",
    746: "LASCARI",
    690: "LICATA",
    703: "SCLAFANI BAGNI",
    700: "MUSSOMELI",
    689: "CANICATTì",
    740: "ALIA",
    687: "CAMMARATA",
    684: "AGRIGENTO MANDRASCAVA",
    685: "ARAGONA",
    755: "TERMINI IMERESE",
    683: "AGRIGENTO SCIBICA",
    747: "MEZZOJUSO",
    748: "MISILMERI",
    686: "BIVONA",
    750: "PALERMO",
    693: "RIBERA",
    745: "CORLEONE",
    749: "MONREALE VIGNA API",
    742: "CAMPOREALE",
    751: "PARTINICO",
    744: "Contessa Entellina",
    773: "CASTELLAMMARE DEL GOLFO",
    774: "CASTELVETRANO",
    778: "SALEMI",
    780: "TRAPANI FULGATORE",
    775: "ERICE",
    779: "TRAPANI FONTANASALSA"
}

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type and extract Year and Season
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['Year'] = df['DATETIME'].dt.year

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Season'] = df['DATETIME'].dt.month.apply(map_month_to_season)

def calculate_hurst_exponent(series):
    # Remove zero and NaN values from the series
    series = series[(series != 0) & (~np.isnan(series))]
    
    if len(series) < 100:
        # If series length is less than 100, return NaN
        return np.nan
    
    try:
        # Calculate Hurst exponent
        H, _, _ = compute_Hc(series, kind='random_walk', simplified=False)
        return H
    except ValueError:
        # If Hurst exponent cannot be computed, return NaN
        return np.nan

# Create a list of seasons
unique_seasons = df['Season'].unique()
n_seasons = len(unique_seasons)

# Calculate the number of rows needed for two plots per row
n_rows = np.ceil(n_seasons / 2).astype(int)

# Initialize figure for plotting with two columns
fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5 * n_rows))
axes = axes.flatten()  # Flatten the axes array for easy iteration

for i, season in enumerate(unique_seasons):
    ax = axes[i]

    # Filter the DataFrame for the current season
    season_df = df[df['Season'] == season]

    # Group the data by station and year, and calculate the Hurst exponent
    hurst_values = season_df.groupby(['STATION', 'Year'])['VALUE'].apply(calculate_hurst_exponent).reset_index()
    hurst_pivot = hurst_values.pivot_table(index='STATION', columns='Year', values='VALUE')

    # Replace NaN with a specific value (e.g., -1) for coloring
    hurst_pivot_filled = hurst_pivot.fillna(-1)

    # Plotting for the current season
    cmap = cm.coolwarm
    c = ax.imshow(hurst_pivot_filled, cmap=cmap, vmin=0.1, vmax=0.7)

    # Set the labels for the stations and years
    ax.set_xticks(np.arange(len(hurst_pivot.columns)))
    ax.set_yticks(np.arange(len(hurst_pivot.index)))

    ax.set_xticklabels(hurst_pivot.columns)
    ax.set_yticklabels([station_order.get(station, "") for station in hurst_pivot.index])

    ax.set_title(f'Hurst Exponent for {season}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Station')

    # Custom color for cells where Hurst exponent couldn't be computed (e.g., -1)
    cmap.set_under('black')  # Use 'black' for values under 0 (e.g., -1)

# If the number of plots is odd, hide the last subplot (if unused)
if n_seasons % 2 != 0:
    axes[-1].axis('off')

# Add a colorbar
#fig.colorbar(c, ax=axes, shrink=0.95, orientation='horizontal')
plt.tight_layout()
plt.show()


In [ ]:
# Create a list of seasons
unique_seasons = df['Season'].unique()
n_seasons = len(unique_seasons)

# Calculate the number of rows needed for two plots per row
n_rows = np.ceil(n_seasons / 2).astype(int)

# Initialize figure for plotting with two columns
fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5 * n_rows))
axes = axes.flatten()  # Flatten the axes array for easy iteration

for i, season in enumerate(unique_seasons):
    ax = axes[i]

    # Filter the DataFrame for the current season
    season_df = df[df['Season'] == season]

    # Group the data by station and year, and calculate the Hurst exponent
    hurst_values = season_df.groupby(['STATION', 'Year'])['VALUE'].apply(calculate_hurst_exponent).reset_index()
    hurst_pivot = hurst_values.pivot_table(index='STATION', columns='Year', values='VALUE')

    # Replace NaN with a specific value (e.g., -1) for coloring
    hurst_pivot_filled = hurst_pivot.fillna(-1)

    # Plotting for the current season
    cmap = cm.coolwarm
    c = ax.imshow(hurst_pivot_filled, cmap=cmap, vmin=0.1, vmax=0.7)

    # Set the labels for the stations and years
    ax.set_xticks(np.arange(len(hurst_pivot.columns)))
    ax.set_yticks(np.arange(len(hurst_pivot.index)))

    ax.set_xticklabels(hurst_pivot.columns)
    ax.set_yticklabels([station_order.get(station, "") for station in hurst_pivot.index])

    ax.set_title(f'Hurst Exponent for {season}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Station')

    # Custom color for cells where Hurst exponent couldn't be computed (e.g., -1)
    cmap.set_under('black')  # Use 'black' for values under 0 (e.g., -1)

# If the number of plots is odd, hide the last subplot (if unused)
if n_seasons % 2 != 0:
    axes[-1].axis('off')

# Add a colorbar
#fig.colorbar(c, ax=axes, shrink=0.95, orientation='horizontal')
plt.tight_layout()
plt.show()


In [ ]:
# mann kendall test - yearly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from matplotlib import cm
import matplotlib.colors as mcolors

# Define the station order dictionary
station_order = {
    729: "MESSINA",
    727: "FIUMEDINISI",
    738: "SAN PIER NICETO",
    739: "TORREGROTTA",
    707: "RIPOSTO",
    764: "SIRACUSA",
    734: "NOVARA DI SICILIA",
    708: "LINGUAGLOSSA",
    769: "PACHINO",
    706: "CATANIA",
    767: "NOTO",
    713: "PEDARA",
    735: "PATTI",
    758: "ISPICA",
    715: "RANDAZZO",
    766: "LENTINI",
    765: "FRANCOFONTE",
    709: "MALETTO",
    770: "PALAZZOLO ACREIDE",
    712: "PATERNò",
    705: "BRONTE",
    733: "NASO",
    711: "MINEO",
    725: "CESARò VIGNAZZA",
    756: "RAGUSA",
    761: "SCICLI",
    730: "MILITELLO ROSMARINO",
    737: "SAN FRATELLO",
    757: "COMISO",
    716: "CALTAGIRONE",
    710: "MAZZARRONE",
    760: "SANTA CROCE CAMERINA",
    723: "CARONIA BUZZA",
    717: "AIDONE",
    721: "NICOSIA",
    762: "ACATE",
    722: "PIAZZA ARMERINA",
    731: "MISTRETTA",
    736: "PETTINEO",
    699: "MAZZARINO",
    752: "GANGI",
    718: "ENNA",
    743: "CASTELBUONO",
    701: "RIESI",
    695: "CALTANISSETTA",
    753: "PETRALIA SOTTANA",
    754: "POLIZZI GENEROSA",
    696: "DELIA",
    746: "LASCARI",
    690: "LICATA",
    703: "SCLAFANI BAGNI",
    700: "MUSSOMELI",
    689: "CANICATTì",
    740: "ALIA",
    687: "CAMMARATA",
    684: "AGRIGENTO MANDRASCAVA",
    685: "ARAGONA",
    755: "TERMINI IMERESE",
    683: "AGRIGENTO SCIBICA",
    747: "MEZZOJUSO",
    748: "MISILMERI",
    686: "BIVONA",
    750: "PALERMO",
    693: "RIBERA",
    745: "CORLEONE",
    749: "MONREALE VIGNA API",
    742: "CAMPOREALE",
    751: "PARTINICO",
    744: "Contessa Entellina",
    773: "CASTELLAMMARE DEL GOLFO",
    774: "CASTELVETRANO",
    778: "SALEMI",
    780: "TRAPANI FULGATORE",
    775: "ERICE",
    779: "TRAPANI FONTANASALSA"
}

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type and extract Year and Season
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['Year'] = df['DATETIME'].dt.year


def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'


df['Season'] = df['DATETIME'].dt.month.apply(map_month_to_season)


def calculate_mann_kendall_test(series):
    # Remove NaN values from the series
    series = series.dropna()

    if len(series) < 10:
        # If series length is less than 10, return NaN
        return np.nan

    try:
        # Perform Mann-Kendall test
        tau, pvalue = kendalltau(np.arange(len(series)), series)
        return pvalue
    except ValueError:
        # If test cannot be performed, return NaN
        return np.nan


def custom_colormap():
    # Define a color dictionary for the range
    cdict = {'red': ((0.0, 0.0, 0.0),  # Blue at 0
                     (0.05, 1.0, 1.0),  # White at 0.05
                     (1.0, 0.0, 0.0)),  # Black at 1

             'green': ((0.0, 0.0, 0.0),
                       (0.05, 1.0, 1.0),  # White at 0.05
                       (1.0, 0.0, 0.0)),  # Black at 1

             'blue': ((0.0, 1.0, 1.0),  # Blue at 0
                      (0.05, 1.0, 1.0),  # White at 0.05
                      (1.0, 0.0, 0.0))}  # Black at 1

    return mcolors.LinearSegmentedColormap('CustomMap', cdict)


cmap = custom_colormap()

# Create a list of seasons
unique_seasons = df['Season'].unique()
n_seasons = len(unique_seasons)
n_rows = np.ceil(n_seasons / 2).astype(int)

# Initialize figure for plotting with two columns
fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5 * n_rows))
axes = axes.flatten()

for i, season in enumerate(unique_seasons):
    ax = axes[i]
    season_df = df[df['Season'] == season]
    mann_kendall_values = season_df.groupby(['STATION', 'Year'])['VALUE'].apply(calculate_mann_kendall_test).reset_index()
    mann_kendall_pivot = mann_kendall_values.pivot_table(index='STATION', columns='Year', values='VALUE')
    mann_kendall_pivot_filled = mann_kendall_pivot.fillna(-1)

    cmap.set_under('black')
    im = ax.imshow(mann_kendall_pivot_filled, cmap=cmap ) #vmin=0, vmax=0.1   
    ax.set_xticks(np.arange(len(mann_kendall_pivot.columns)))
    ax.set_yticks(np.arange(len(mann_kendall_pivot.index)))
    ax.set_xticklabels(mann_kendall_pivot.columns, rotation=45, fontsize=8)
    ax.set_yticklabels([station_order.get(station, "") for station in mann_kendall_pivot.index], fontsize=8)
    ax.set_title(f'Mann-Kendall Test P-values for {season}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Station')

    # Annotating each cell with the p-value
    #for (j, k), val in np.ndenumerate(mann_kendall_pivot_filled):
        #if val != -1:  # Exclude the missing data indicator
           # ax.text(k, j, f'{val:.2f}', ha='center', va='center', color='white')

if n_seasons % 2 != 0:
    axes[-1].axis('off')  # Hide the last subplot if odd number of seasons

cbar = fig.colorbar(im, ax=axes, orientation='horizontal', fraction=0.04, pad=0.04)
cbar.ax.xaxis.set_ticks_position('top')  # Position ticks at the top
cbar.ax.xaxis.set_label_position('top')  # Position label at the top
plt.tight_layout()
plt.show()

In [ ]:
# mann kendall test - 2 years period 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
from matplotlib import cm
import matplotlib.colors as mcolors

# Define the station order dictionary
station_order = {
    729: "MESSINA",
    727: "FIUMEDINISI",
    738: "SAN PIER NICETO",
    739: "TORREGROTTA",
    707: "RIPOSTO",
    764: "SIRACUSA",
    734: "NOVARA DI SICILIA",
    708: "LINGUAGLOSSA",
    769: "PACHINO",
    706: "CATANIA",
    767: "NOTO",
    713: "PEDARA",
    735: "PATTI",
    758: "ISPICA",
    715: "RANDAZZO",
    766: "LENTINI",
    765: "FRANCOFONTE",
    709: "MALETTO",
    770: "PALAZZOLO ACREIDE",
    712: "PATERNò",
    705: "BRONTE",
    733: "NASO",
    711: "MINEO",
    725: "CESARò VIGNAZZA",
    756: "RAGUSA",
    761: "SCICLI",
    730: "MILITELLO ROSMARINO",
    737: "SAN FRATELLO",
    757: "COMISO",
    716: "CALTAGIRONE",
    710: "MAZZARRONE",
    760: "SANTA CROCE CAMERINA",
    723: "CARONIA BUZZA",
    717: "AIDONE",
    721: "NICOSIA",
    762: "ACATE",
    722: "PIAZZA ARMERINA",
    731: "MISTRETTA",
    736: "PETTINEO",
    699: "MAZZARINO",
    752: "GANGI",
    718: "ENNA",
    743: "CASTELBUONO",
    701: "RIESI",
    695: "CALTANISSETTA",
    753: "PETRALIA SOTTANA",
    754: "POLIZZI GENEROSA",
    696: "DELIA",
    746: "LASCARI",
    690: "LICATA",
    703: "SCLAFANI BAGNI",
    700: "MUSSOMELI",
    689: "CANICATTì",
    740: "ALIA",
    687: "CAMMARATA",
    684: "AGRIGENTO MANDRASCAVA",
    685: "ARAGONA",
    755: "TERMINI IMERESE",
    683: "AGRIGENTO SCIBICA",
    747: "MEZZOJUSO",
    748: "MISILMERI",
    686: "BIVONA",
    750: "PALERMO",
    693: "RIBERA",
    745: "CORLEONE",
    749: "MONREALE VIGNA API",
    742: "CAMPOREALE",
    751: "PARTINICO",
    744: "Contessa Entellina",
    773: "CASTELLAMMARE DEL GOLFO",
    774: "CASTELVETRANO",
    778: "SALEMI",
    780: "TRAPANI FULGATORE",
    775: "ERICE",
    779: "TRAPANI FONTANASALSA"
}

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type and extract Year and Season
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['Year'] = df['DATETIME'].dt.year

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Season'] = df['DATETIME'].dt.month.apply(map_month_to_season)

# Function to assign a year span based on the year and season
def assign_year_span(year, season):
    if year % 2 == 0:  # Even year
        return f'{year}-{year+1}'
    else:  # Odd year
        return f'{year-1}-{year}'

df['YearSpan'] = df.apply(lambda row: assign_year_span(row['Year'], row['Season']), axis=1)

def calculate_mann_kendall_test(series):
    # Remove NaN values from the series
    series = series.dropna()

    if len(series) < 10:
        # If series length is less than 10, return NaN
        return np.nan

    try:
        # Perform Mann-Kendall test
        tau, pvalue = kendalltau(np.arange(len(series)), series)
        return pvalue
    except ValueError:
        # If test cannot be performed, return NaN
        return np.nan

def custom_colormap():
    # Define a color dictionary for the range
    cdict = {'red': ((0.0, 0.0, 0.0),  # Blue at 0
                     (0.05, 1.0, 1.0),  # White at 0.05
                     (1.0, 0.0, 0.0)),  # Black at 1

             'green': ((0.0, 0.0, 0.0),
                       (0.05, 1.0, 1.0),  # White at 0.05
                       (1.0, 0.0, 0.0)),  # Black at 1

             'blue': ((0.0, 1.0, 1.0),  # Blue at 0
                      (0.05, 1.0, 1.0),  # White at 0.05
                      (1.0, 0.0, 0.0))}  # Black at 1

    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

cmap = custom_colormap()

# Create a list of seasons
unique_seasons = df['Season'].unique()
n_seasons = len(unique_seasons)
n_rows = np.ceil(n_seasons / 2).astype(int)

# Initialize figure for plotting with two columns
fig, axes = plt.subplots(n_rows, 2, figsize=(15, 7 * n_rows))
axes = axes.flatten()

for i, season in enumerate(unique_seasons):
    ax = axes[i]
    season_df = df[df['Season'] == season]
    mann_kendall_values = season_df.groupby(['STATION', 'YearSpan'])['VALUE'].apply(calculate_mann_kendall_test).reset_index()
    mann_kendall_pivot = mann_kendall_values.pivot_table(index='STATION', columns='YearSpan', values='VALUE')
    mann_kendall_pivot_filled = mann_kendall_pivot.fillna(-1)

    cmap.set_under('black')
    im = ax.imshow(mann_kendall_pivot_filled, cmap=cmap)
    ax.set_xticks(np.arange(len(mann_kendall_pivot.columns)))
    ax.set_yticks(np.arange(len(mann_kendall_pivot.index)))
    ax.set_xticklabels(mann_kendall_pivot.columns, rotation=45, fontsize=8)
    ax.set_yticklabels([station_order.get(station, "") for station in mann_kendall_pivot.index], fontsize=8)
    ax.set_title(f'Mann-Kendall Test P-values for {season} (2-year periods)')
    ax.set_xlabel('Year Span')
    ax.set_ylabel('Station')

    # Optional: Annotating each cell with the p-value
    # for (j, k), val in np.ndenumerate(mann_kendall_pivot_filled):
    #    if val != -1:  # Exclude the missing data indicator
    #        ax.text(k, j, f'{val:.2f}', ha='center', va='center', color='white')

if n_seasons % 2 != 0:
    axes[-1].axis('off')  # Hide the last subplot if odd number of seasons

cbar = fig.colorbar(im, ax=axes, orientation='horizontal', fraction=0.04, pad=0.04)
cbar.ax.xaxis.set_ticks_position('top')  # Position ticks at the top
cbar.ax.xaxis.set_label_position('top')  # Position label at the top
plt.tight_layout()
plt.show()


In [ ]:
#3d yearly volumes
#OKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKKK
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PA",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}

# Specify the stations you want to label
stations_to_label = ["CT", "PA", "ME", "AG", "EN", "CL", "TP", "SR", "RG"]

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

df['Year'] = df['DATETIME'].dt.year

# Define a colormap
cmap = cm.colors.ListedColormap(['#9ABDDC', '#B4CF68', '#FFD872', '#FF96C5', '#FF00FF', 'purple'])

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')  # Create a single 3D subplot

filtered_df = df[(df['VALUE'] > 1) & (df['VALUE'] < 2000)]
pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='max').fillna(0)

# Reorder the pivot_table and Y_labels based on station_order
pivot_table = pivot_table.loc[station_ids_in_order]

# Create a list of y-axis labels
Y_labels = []
for station_id in station_ids_in_order:
    station_name = station_order[station_id]
    if station_name.upper() in stations_to_label:
        Y_labels.append(station_name)  # Label specific cities
    else:
        Y_labels.append("")  # Empty string for other stations

# Calculate the normalization for the data
norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())

# Apply the normalization to the count values and map to the 'coolwarm' colormap
colors = cmap(norm(pivot_table.values.ravel()))

Z = pivot_table.values
X_labels = pivot_table.columns[::1]

X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))

dx = np.ones(Z.shape) * 0.75
dy = np.ones(Z.shape) * 0.75
dz = Z

for x, y, z, color in zip(X.flatten(), Y.flatten(), dz.flatten(), colors):
    ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color)

ax.set_title('Yearly Sum Volumes')
ax.set_zlabel('mm')
#ax.set_zlim(0, 500)
ax.set_yticks(np.arange(len(Y_labels)) + 5.5)
ax.set_yticklabels(Y_labels, fontsize= 7)
ax.set_xticks(np.arange(len(X_labels)))
ax.set_xticklabels(X_labels, rotation=45)

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
#fig.savefig("75gauge_3d_yearly_volumes.jpg", dpi=300)
plt.show()

In [ ]:
#2d yearly volumes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    729: "MESSINA",
    727: "FIUMEDINISI",
    738: "SAN PIER NICETO",
    739: "TORREGROTTA",
    707: "RIPOSTO",
    764: "SIRACUSA",
    734: "NOVARA DI SICILIA",
    708: "LINGUAGLOSSA",
    769: "PACHINO",
    706: "CATANIA",
    767: "NOTO",
    713: "PEDARA",
    735: "PATTI",
    758: "ISPICA",
    715: "RANDAZZO",
    766: "LENTINI",
    765: "FRANCOFONTE",
    709: "MALETTO",
    770: "PALAZZOLO ACREIDE",
    712: "PATERNò",
    705: "BRONTE",
    733: "NASO",
    711: "MINEO",
    725: "CESARò VIGNAZZA",
    756: "RAGUSA",
    761: "SCICLI",
    730: "MILITELLO ROSMARINO",
    737: "SAN FRATELLO",
    757: "COMISO",
    716: "CALTAGIRONE",
    710: "MAZZARRONE",
    760: "SANTA CROCE CAMERINA",
    723: "CARONIA BUZZA",
    717: "AIDONE",
    721: "NICOSIA",
    762: "ACATE",
    722: "PIAZZA ARMERINA",
    731: "MISTRETTA",
    736: "PETTINEO",
    699: "MAZZARINO",
    752: "GANGI",
    718: "ENNA",
    743: "CASTELBUONO",
    701: "RIESI",
    695: "CALTANISSETTA",
    753: "PETRALIA SOTTANA",
    754: "POLIZZI GENEROSA",
    696: "DELIA",
    746: "LASCARI",
    690: "LICATA",
    703: "SCLAFANI BAGNI",
    700: "MUSSOMELI",
    689: "CANICATTì",
    740: "ALIA",
    687: "CAMMARATA",
    684: "AGRIGENTO MANDRASCAVA",
    685: "ARAGONA",
    755: "TERMINI IMERESE",
    683: "AGRIGENTO SCIBICA",
    747: "MEZZOJUSO",
    748: "MISILMERI",
    686: "BIVONA",
    750: "PALERMO",
    693: "RIBERA",
    745: "CORLEONE",
    749: "MONREALE VIGNA API",
    742: "CAMPOREALE",
    751: "PARTINICO",
    744: "Contessa Entellina",
    773: "CASTELLAMMARE DEL GOLFO",
    774: "CASTELVETRANO",
    778: "SALEMI",
    780: "TRAPANI FULGATORE",
    775: "ERICE",
    779: "TRAPANI FONTANASALSA"
}

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

df['Year'] = df['DATETIME'].dt.year

# Define a custom colormap with specific colors from 'tab10'
cmap = cm.colors.ListedColormap(['#9ABDDC','#B4CF68','#FFD872', '#FF96C5','#FF00FF','purple'])

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

# Create a single plot for yearly data per station
fig, ax = plt.subplots(figsize=(10, 6))

filtered_df = df[(df['VALUE'] > 1) & (df['VALUE'] < 2000)]

pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='max').fillna(0)

# Reorder the pivot_table based on station_order
pivot_table = pivot_table.loc[station_ids_in_order]

# Create a list of y-axis labels
Y_labels = [station_order[station_id] for station_id in station_ids_in_order]

# Calculate the normalization for the data
norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())

# Apply the normalization to the count values and map to the custom colormap
colors = cmap(norm(pivot_table.values))

# Create X and Y data for the heatmap
X_labels = pivot_table.columns
X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))

# Plot the heatmap
heatmap = ax.imshow(pivot_table.values, cmap=cmap, norm=norm)
ax.set_title('Yearly Data per Station')
ax.set_xlabel('Year')
ax.set_ylabel('Station')
ax.set_xticks(np.arange(len(X_labels)))
ax.set_xticklabels(X_labels, rotation=90, fontsize= 5)
ax.set_yticks(np.arange(len(Y_labels)))
ax.set_yticklabels(Y_labels, rotation=0 , fontsize= 5)

# Add colorbar
cbar = plt.colorbar(heatmap, ax=ax)
cbar.set_label('mm')


plt.tight_layout(rect=[0, 0.05, 1, 0.95])
#fig.savefig("75gauge_3d_yearlyvolumessheatmap.jpg", dpi=300)

plt.show()

In [ ]:
#3d seasonal volumes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PA",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}

# Specify the stations you want to label
stations_to_label = ["CT", "PA", "ME", "AG", "EN", "CL", "TP", "SR", "RG"]

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Define a colormap
#cmap = cm.get_cmap('Paired',6)
cmap = cm.colors.ListedColormap(['#9ABDDC','#B4CF68','#FFD872', '#FF96C5','#FF00FF','purple'])

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

unique_seasons = df['Season'].unique()

fig = plt.figure(figsize=(22, 22))

for i, season in enumerate(unique_seasons, 1):
    ax = fig.add_subplot(2, 2, i, projection='3d')
    
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='sum').fillna(0)
    
    # Reorder the pivot_table and Y_labels based on station_order
    pivot_table = pivot_table.loc[station_ids_in_order]
    
    # Create a list of y-axis labels
    Y_labels = []
    for station_id in station_ids_in_order:
        station_name = station_order[station_id]
        if station_name.upper() in stations_to_label:
            Y_labels.append(station_name)  # Label specific cities
        else:
            Y_labels.append("")  # Empty string for other stations
    
    # Calculate the normalization for this season's data
    norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())
    
    # Apply the normalization to the count values and map to the 'coolwarm' colormap
    colors = cmap(norm(pivot_table.values.ravel()))

    Z = pivot_table.values
    X_labels = pivot_table.columns[::1]
    
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    dx = np.ones(Z.shape) * 0.75
    dy = np.ones(Z.shape) * 0.75
    dz = Z
    
    for x, y, z, color in zip(X.flatten(), Y.flatten(), dz.flatten(), colors):
        #ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color, edgecolor='k')
        ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color)

    ax.set_title(f'Volumes {season}')
    ax.set_zlabel('mm')
    ax.set_zlim(0, 1000)
    ax.set_yticks(np.arange(len(Y_labels))+7.5)
    ax.set_yticklabels(Y_labels)
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=45)

plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
#fig.savefig("75gauge_3d_volumes.jpg", dpi=300)
plt.show()

In [ ]:
# Define a colormap
cmap = cm.colors.ListedColormap(['#9ABDDC', '#B4CF68', '#FFD872', '#FF96C5', '#FF00FF', 'purple'])

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

unique_seasons = df['Season'].unique()

# Loop through each season and save the plot separately
for i, season in enumerate(unique_seasons):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='sum').fillna(0)
    
    # Reorder the pivot_table and Y_labels based on station_order
    pivot_table = pivot_table.loc[station_ids_in_order]
    
    # Create a list of y-axis labels
    Y_labels = []
    for station_id in station_ids_in_order:
        station_name = station_order[station_id]
        if station_name.upper() in stations_to_label:
            Y_labels.append(station_name)  # Label specific cities
        else:
            Y_labels.append("")  # Empty string for other stations
    
    # Calculate the normalization for this season's data
    norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())
    
    # Apply the normalization to the count values and map to the colormap
    colors = cmap(norm(pivot_table.values.ravel()))

    Z = pivot_table.values
    X_labels = pivot_table.columns
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    dx = np.ones(Z.shape) * 0.75
    dy = np.ones(Z.shape) * 0.75
    dz = Z
    
    for x, y, z, color in zip(X.flatten(), Y.flatten(), dz.flatten(), colors):
        ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color)
    
    ax.set_title(f'Volumes {season}', fontsize=20)
    ax.set_zlabel('mm')
    ax.set_zlim(0, 1000)
    ax.set_yticks(np.arange(len(Y_labels)))
    ax.set_yticklabels(Y_labels)
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=45)
    
    # Save each plot as a separate file
    fig.savefig(f"75gauge_3d_volumes_{season}.jpg", dpi=500)
    plt.close(fig)  # Close the figure to avoid overlapping plots


In [ ]:
#2d seasonal volumes

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    729: "MESSINA",
    727: "FIUMEDINISI",
    738: "SAN PIER NICETO",
    739: "TORREGROTTA",
    707: "RIPOSTO",
    764: "SIRACUSA",
    734: "NOVARA DI SICILIA",
    708: "LINGUAGLOSSA",
    769: "PACHINO",
    706: "CATANIA",
    767: "NOTO",
    713: "PEDARA",
    735: "PATTI",
    758: "ISPICA",
    715: "RANDAZZO",
    766: "LENTINI",
    765: "FRANCOFONTE",
    709: "MALETTO",
    770: "PALAZZOLO ACREIDE",
    712: "PATERNò",
    705: "BRONTE",
    733: "NASO",
    711: "MINEO",
    725: "CESARò VIGNAZZA",
    756: "RAGUSA",
    761: "SCICLI",
    730: "MILITELLO ROSMARINO",
    737: "SAN FRATELLO",
    757: "COMISO",
    716: "CALTAGIRONE",
    710: "MAZZARRONE",
    760: "SANTA CROCE CAMERINA",
    723: "CARONIA BUZZA",
    717: "AIDONE",
    721: "NICOSIA",
    762: "ACATE",
    722: "PIAZZA ARMERINA",
    731: "MISTRETTA",
    736: "PETTINEO",
    699: "MAZZARINO",
    752: "GANGI",
    718: "ENNA",
    743: "CASTELBUONO",
    701: "RIESI",
    695: "CALTANISSETTA",
    753: "PETRALIA SOTTANA",
    754: "POLIZZI GENEROSA",
    696: "DELIA",
    746: "LASCARI",
    690: "LICATA",
    703: "SCLAFANI BAGNI",
    700: "MUSSOMELI",
    689: "CANICATTì",
    740: "ALIA",
    687: "CAMMARATA",
    684: "AGRIGENTO MANDRASCAVA",
    685: "ARAGONA",
    755: "TERMINI IMERESE",
    683: "AGRIGENTO SCIBICA",
    747: "MEZZOJUSO",
    748: "MISILMERI",
    686: "BIVONA",
    750: "PALERMO",
    693: "RIBERA",
    745: "CORLEONE",
    749: "MONREALE VIGNA API",
    742: "CAMPOREALE",
    751: "PARTINICO",
    744: "Contessa Entellina",
    773: "CASTELLAMMARE DEL GOLFO",
    774: "CASTELVETRANO",
    778: "SALEMI",
    780: "TRAPANI FULGATORE",
    775: "ERICE",
    779: "TRAPANI FONTANASALSA"
}

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Define a colormap
# Define a custom colormap with specific colors from 'tab10'
cmap = cm.colors.ListedColormap(['#9ABDDC','#B4CF68','#FFD872', '#FF96C5','#FF00FF','purple'])
#cmap = cm.get_cmap('Paired',6)
#'#FFAAB0'

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

unique_seasons = df['Season'].unique()

# Calculate the number of rows and columns for subplots based on the number of seasons
num_seasons = len(unique_seasons)
num_rows = int(np.ceil(num_seasons / 2))
num_cols = 2

fig, axes = plt.subplots(num_rows, num_cols, figsize=(22, 22))

for i, season in enumerate(unique_seasons, 1):
    row_index = (i - 1) // num_cols
    col_index = (i - 1) % num_cols
    ax = axes[row_index, col_index]
    
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='sum').fillna(0)
    
    # Reorder the pivot_table based on station_order
    pivot_table = pivot_table.loc[station_ids_in_order]
    
    # Create a list of y-axis labels
    Y_labels = [station_order[station_id] for station_id in station_ids_in_order]
    
    # Calculate the normalization for this season's data
    norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())
    
    # Apply the normalization to the count values and map to the 'coolwarm' colormap
    colors = cmap(norm(pivot_table.values))
    
    # Create X and Y data for the heatmap
    X_labels = pivot_table.columns[::1]
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    # Plot the heatmap
    heatmap = ax.imshow(pivot_table.values, cmap=cmap, norm=norm)
    ax.set_title(f'Volumes {season}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Station')
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=90)
    ax.set_yticks(np.arange(len(Y_labels)))
    ax.set_yticklabels(Y_labels, rotation=0)
    
    # Add colorbar
    cbar = plt.colorbar(heatmap, ax=ax)
    cbar.set_label('mm')

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
fig.savefig("75gauge_3d_volumesheatmap.jpg", dpi=300)

plt.show()

In [ ]:
#3d seasonal counts 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PA",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}

# Specify the stations you want to label
stations_to_label = ["CT", "PA", "ME", "AG", "EN", "CL", "TP", "SR", "RG"]

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Define a colormap
#cmap = cm.get_cmap('Paired',6)
cmap = cm.colors.ListedColormap(['#9ABDDC','#B4CF68','#FFD872', '#FF96C5','#FF00FF','purple'])

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

unique_seasons = df['Season'].unique()

fig = plt.figure(figsize=(22, 22))

for i, season in enumerate(unique_seasons, 1):
    ax = fig.add_subplot(2, 2, i, projection='3d')
    
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='count').fillna(0)
    
    # Reorder the pivot_table and Y_labels based on station_order
    pivot_table = pivot_table.loc[station_ids_in_order]
    
    # Create a list of y-axis labels
    Y_labels = []
    for station_id in station_ids_in_order:
        station_name = station_order[station_id]
        if station_name.upper() in stations_to_label:
            Y_labels.append(station_name)  # Label specific cities
        else:
            Y_labels.append("")  # Empty string for other stations
    
    # Calculate the normalization for this season's data
    norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())
    
    # Apply the normalization to the count values and map to the 'coolwarm' colormap
    colors = cmap(norm(pivot_table.values.ravel()))

    Z = pivot_table.values
    X_labels = pivot_table.columns[::1]
    
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    dx = np.ones(Z.shape) * 0.75
    dy = np.ones(Z.shape) * 0.75
    dz = Z
    
    for x, y, z, color in zip(X.flatten(), Y.flatten(), dz.flatten(), colors):
        #ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color, edgecolor='k')
        ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color)

    ax.set_title(f'Counts {season}')
    ax.set_zlabel('h')
    ax.set_zlim(0, 300)
    ax.set_yticks(np.arange(len(Y_labels))+7.5)
    ax.set_yticklabels(Y_labels)
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=45)

plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
fig.savefig("75gauge_3d_counts.jpg", dpi=300)
plt.show()

In [ ]:
#2d seasonal counts
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
# Define the station order dictionary
station_order = {
    729: "MESSINA",
    727: "FIUMEDINISI",
    738: "SAN PIER NICETO",
    739: "TORREGROTTA",
    707: "RIPOSTO",
    764: "SIRACUSA",
    734: "NOVARA DI SICILIA",
    708: "LINGUAGLOSSA",
    769: "PACHINO",
    706: "CATANIA",
    767: "NOTO",
    713: "PEDARA",
    735: "PATTI",
    758: "ISPICA",
    715: "RANDAZZO",
    766: "LENTINI",
    765: "FRANCOFONTE",
    709: "MALETTO",
    770: "PALAZZOLO ACREIDE",
    712: "PATERNò",
    705: "BRONTE",
    733: "NASO",
    711: "MINEO",
    725: "CESARò VIGNAZZA",
    756: "RAGUSA",
    761: "SCICLI",
    730: "MILITELLO ROSMARINO",
    737: "SAN FRATELLO",
    757: "COMISO",
    716: "CALTAGIRONE",
    710: "MAZZARRONE",
    760: "SANTA CROCE CAMERINA",
    723: "CARONIA BUZZA",
    717: "AIDONE",
    721: "NICOSIA",
    762: "ACATE",
    722: "PIAZZA ARMERINA",
    731: "MISTRETTA",
    736: "PETTINEO",
    699: "MAZZARINO",
    752: "GANGI",
    718: "ENNA",
    743: "CASTELBUONO",
    701: "RIESI",
    695: "CALTANISSETTA",
    753: "PETRALIA SOTTANA",
    754: "POLIZZI GENEROSA",
    696: "DELIA",
    746: "LASCARI",
    690: "LICATA",
    703: "SCLAFANI BAGNI",
    700: "MUSSOMELI",
    689: "CANICATTì",
    740: "ALIA",
    687: "CAMMARATA",
    684: "AGRIGENTO MANDRASCAVA",
    685: "ARAGONA",
    755: "TERMINI IMERESE",
    683: "AGRIGENTO SCIBICA",
    747: "MEZZOJUSO",
    748: "MISILMERI",
    686: "BIVONA",
    750: "PALERMO",
    693: "RIBERA",
    745: "CORLEONE",
    749: "MONREALE VIGNA API",
    742: "CAMPOREALE",
    751: "PARTINICO",
    744: "Contessa Entellina",
    773: "CASTELLAMMARE DEL GOLFO",
    774: "CASTELVETRANO",
    778: "SALEMI",
    780: "TRAPANI FULGATORE",
    775: "ERICE",
    779: "TRAPANI FONTANASALSA"
}

# Specify the stations you want to label
stations_to_label = ["CATANIA", "PALERMO", "MESSINA", "AGRIGENTO SCIBICA", "ENNA", "CALTANISSETTA", "TRAPANI FONTANASALSA", "SIRACUSA", "RAGUSA"]

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Define a colormap
# Define a custom colormap with specific colors from 'tab10'
cmap = cm.colors.ListedColormap(['#9ABDDC','#B4CF68','#FFD872', '#FF96C5','#FF00FF','purple'])
#cmap = cm.get_cmap('Paired',6)
#'#FFAAB0'

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

unique_seasons = df['Season'].unique()

# Calculate the number of rows and columns for subplots based on the number of seasons
num_seasons = len(unique_seasons)
num_rows = int(np.ceil(num_seasons / 2))
num_cols = 2

fig, axes = plt.subplots(num_rows, num_cols, figsize=(22, 22))

for i, season in enumerate(unique_seasons, 1):
    row_index = (i - 1) // num_cols
    col_index = (i - 1) % num_cols
    ax = axes[row_index, col_index]
    
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='count').fillna(0)
    
    # Reorder the pivot_table based on station_order
    pivot_table = pivot_table.loc[station_ids_in_order]
    
    # Create a list of y-axis labels
    Y_labels = [station_order[station_id] for station_id in station_ids_in_order]
    
    # Calculate the normalization for this season's data
    norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())
    
    # Apply the normalization to the count values and map to the 'coolwarm' colormap
    colors = cmap(norm(pivot_table.values))
    
    # Create X and Y data for the heatmap
    X_labels = pivot_table.columns[::1]
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    # Plot the heatmap
    heatmap = ax.imshow(pivot_table.values, cmap=cmap, norm=norm)
    ax.set_title(f'Counts {season}')
    ax.set_xlabel('Year')
    ax.set_ylabel('Station')
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=90)
    ax.set_yticks(np.arange(len(Y_labels)))
    ax.set_yticklabels(Y_labels, rotation=0)
    
    # Add colorbar
    cbar = plt.colorbar(heatmap, ax=ax)
    cbar.set_label('h')

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
fig.savefig("75gauge_3d_countsheatmap.jpg", dpi=300)

plt.show()

In [ ]:
#3d seasonal max/h mm

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize
from matplotlib import cm

# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PA",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}

# Specify the stations you want to label
stations_to_label = ["CT", "PA", "ME", "AG", "EN", "CL", "TP", "SR", "RG"]

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

# Define a colormap
#cmap = cm.get_cmap('Paired',6)
cmap = cm.colors.ListedColormap(['#9ABDDC','#B4CF68','#FFD872', '#FF96C5','#FF00FF','purple'])

# Create a list of station IDs in the desired order
station_ids_in_order = list(station_order.keys())

unique_seasons = df['Season'].unique()

fig = plt.figure(figsize=(22, 22))

for i, season in enumerate(unique_seasons, 1):
    ax = fig.add_subplot(2, 2, i, projection='3d')
    
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='max').fillna(0)
    
    # Reorder the pivot_table and Y_labels based on station_order
    pivot_table = pivot_table.loc[station_ids_in_order]
    
    # Create a list of y-axis labels
    Y_labels = []
    for station_id in station_ids_in_order:
        station_name = station_order[station_id]
        if station_name.upper() in stations_to_label:
            Y_labels.append(station_name)  # Label specific cities
        else:
            Y_labels.append("")  # Empty string for other stations
    
    # Calculate the normalization for this season's data
    norm = Normalize(vmin=pivot_table.values.min(), vmax=pivot_table.values.max())
    
    # Apply the normalization to the count values and map to the 'coolwarm' colormap
    colors = cmap(norm(pivot_table.values.ravel()))

    Z = pivot_table.values
    X_labels = pivot_table.columns[::1]
    
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    dx = np.ones(Z.shape) * 0.75
    dy = np.ones(Z.shape) * 0.75
    dz = Z
    
    for x, y, z, color in zip(X.flatten(), Y.flatten(), dz.flatten(), colors):
        #ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color, edgecolor='k')
        ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color)

    ax.set_title(f'Max/h {season}')
    ax.set_zlabel('mm')
    ax.set_zlim(0, 100)
    ax.set_yticks(np.arange(len(Y_labels))+7.5)
    ax.set_yticklabels(Y_labels)
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=45)

plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
fig.savefig("75gauge_3d_max.jpg", dpi=300)
plt.show()

In [ ]:
#3d seasonal max/h - month of occurrence

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize, ListedColormap
from matplotlib import cm


# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PL",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}

# Specify the stations you want to label
stations_to_label = ["CT", "PA", "ME", "AG", "EN", "CL", "TP", "SR", "RG"]

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type
df['DATETIME'] = pd.to_datetime(df['DATETIME'])

def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month
df['Season'] = df['Month'].apply(map_month_to_season)

#cmap = ListedColormap(plt.cm.get_cmap('tab20', 12).colors[:12])
#colors = cmap.colors
colors = ['#17becf', '#ccccff', '#fee6ed', '#bc8f8f', '#E44B8D', '#fdefb2', '#FFB347', '#b01a00','#cee2cd', '#a0522d','#608da2','white']

#colors = ['#CBF1FA', '#6BA7CC', '#fee6ed', '#fcb4ca', '#E44B8D', '#FFF200', '#FFB347', '#D24E01',
          #'#9DC183', '#2E8B57','#828E84','#F8F8FF']

cmap = ListedColormap(colors)

def get_surface_data_for_season(df, season):
    filtered_df = df[(df['Season'] == season) & (df['VALUE'] > 1) & (df['VALUE'] < 2000)]
    pivot_table = filtered_df.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='max').fillna(0)
    
    # Reorder the pivot_table and Y_labels based on station_order
    pivot_table = pivot_table.loc[station_order.keys()]
    
    Y = pivot_table.index.tolist()
    X_labels = pivot_table.columns[::1]
    
    month_colors = []
    for year in X_labels:
        for station in pivot_table.index:
            series = filtered_df[(filtered_df['STATION'] == station) & (filtered_df['Year'] == year)]['VALUE']
            if not series.empty:
                idx_max = series.idxmax()
                month_of_max = filtered_df.loc[idx_max, 'Month']
                month_colors.append(colors[month_of_max - 1])
            else:
                # Skip plotting for missing data
                continue

    return pivot_table.values, Y, X_labels, month_colors

unique_seasons = df['Season'].unique()

fig = plt.figure(figsize=(22, 22))

for i, season in enumerate(unique_seasons, 1):
    ax = fig.add_subplot(2, 2, i, projection='3d')
    
    Z, Y_labels, X_labels, month_colors = get_surface_data_for_season(df, season)
    X, Y = np.meshgrid(np.arange(len(X_labels)), np.arange(len(Y_labels)))
    
    dx = np.ones(Z.shape) * 0.75
    dy = np.ones(Z.shape) * 0.75
    dz = Z
    
    for x, y, z, color in zip(X.flatten(), Y.flatten(), dz.flatten(), month_colors):
        ax.bar3d(x, y, 0, dx[0, 0], dy[0, 0], z, shade=True, color=color, alpha=0.7, )
    
    ax.set_title(f'max/h {season}')
    ax.set_zlabel('mm')
    ax.set_zlim(0,180)
    ax.set_yticks(np.arange(len(Y_labels)) + 0.5)
    ax.set_yticklabels(Y_labels)
    ax.set_xticks(np.arange(len(X_labels)))
    ax.set_xticklabels(X_labels, rotation=45)

# Define month names
month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Custom Legend for Months
legend_elements = [plt.Line2D([0], [0], color=c, label=month_names[i], marker='o', markersize=20, linestyle='') 
                   for i, c in enumerate(colors)]

fig.legend(handles=legend_elements, loc='right', ncol=1, fontsize= 18 )

plt.tight_layout(rect=[0, 0.05, 1, 0.95]) 
fig.savefig("75gauge_3d_max.jpg", dpi=300)# Adjust the space to avoid overlap
plt.show()

In [ ]:
#2d seasonal max/h - month of occurrence 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize, ListedColormap
from matplotlib import cm
import matplotlib.colors as mcolors 
# Define the station order dictionary
station_order = {
    779: "TP",
    775: "ERICE",
    780: "TRAPANI FULGATORE",
    778: "SALEMI",
    774: "CASTELVETRANO",
    773: "CASTELLAMMARE DEL GOLFO",
    744: "Contessa Entellina",
    751: "PARTINICO",
    742: "CAMPOREALE",
    749: "MONREALE VIGNA API",
    745: "CORLEONE",
    693: "RIBERA",
    750: "PL",
    686: "BIVONA",
    748: "MISILMERI",
    747: "MEZZOJUSO",
    683: "AG",
    755: "TERMINI IMERESE",
    685: "ARAGONA",
    684: "AGRIGENTO MANDRASCAVA",
    687: "CAMMARATA",
    740: "ALIA",
    689: "CANICATTì",
    700: "MUSSOMELI",
    703: "SCLAFANI BAGNI",
    690: "LICATA",
    746: "LASCARI",
    696: "DELIA",
    754: "POLIZZI GENEROSA",
    753: "PETRALIA SOTTANA",
    695: "CL",
    701: "RIESI",
    743: "CASTELBUONO",
    718: "EN",
    752: "GANGI",
    699: "MAZZARINO",
    736: "PETTINEO",
    731: "MISTRETTA",
    722: "PIAZZA ARMERINA",
    762: "ACATE",
    721: "NICOSIA",
    717: "AIDONE",
    723: "CARONIA BUZZA",
    760: "SANTA CROCE CAMERINA",
    710: "MAZZARRONE",
    716: "CALTAGIRONE",
    757: "COMISO",
    737: "SAN FRATELLO",
    730: "MILITELLO ROSMARINO",
    761: "SCICLI",
    756: "RG",
    725: "CESARò VIGNAZZA",
    711: "MINEO",
    733: "NASO",
    705: "BRONTE",
    712: "PATERNò",
    770: "PALAZZOLO ACREIDE",
    709: "MALETTO",
    765: "FRANCOFONTE",
    766: "LENTINI",
    715: "RANDAZZO",
    758: "ISPICA",
    735: "PATTI",
    713: "PEDARA",
    767: "NOTO",
    706: "CT",
    769: "PACHINO",
    708: "LINGUAGLOSSA",
    734: "NOVARA DI SICILIA",
    764: "SR",
    707: "RIPOSTO",
    739: "TORREGROTTA",
    738: "SAN PIER NICETO",
    727: "FIUMEDINISI",
    729: "ME"
}

# Read the CSV file into a DataFrame
df = pd.read_csv('75gauges.csv')

# Convert the 'DATETIME' column to a datetime type and add the 'Year', 'Month', and 'Season' columns
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
df['Year'] = df['DATETIME'].dt.year
df['Month'] = df['DATETIME'].dt.month

# Define the function to map month to season
def map_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Autumn'
    else:
        return 'Winter'

df['Season'] = df['Month'].apply(map_month_to_season)

# Define colors for each month
colors = ['#17becf', '#ccccff', '#fee6ed', '#bc8f8f', '#E44B8D', '#fdefb2', '#FFB347', '#b01a00',
          '#cee2cd', '#a0522d', '#608da2', 'white']

# Create the heatmap for each season
unique_seasons = df['Season'].unique()
seasonal_heatmaps = {}

for season in unique_seasons:
    # Filter data for the season
    season_data = df[df['Season'] == season]

    # Create a pivot table with maximum values for each station and year
    pivot_table = season_data.pivot_table(index='STATION', columns='Year', values='VALUE', aggfunc='max')

    # Reorder the pivot_table based on station_order
    pivot_table = pivot_table.loc[station_order.keys()].fillna(0)

    # Create a 2D array to store the colors
    color_grid = np.zeros((pivot_table.shape[0], pivot_table.shape[1], 3))

    # Assign colors based on the month of maximum value
    for i, station in enumerate(pivot_table.index):
        for j, year in enumerate(pivot_table.columns):
            # Find the month of the maximum value
            max_month_row = season_data[(season_data['STATION'] == station) & (season_data['Year'] == year)]
            max_month_row = max_month_row[max_month_row['VALUE'] == max_month_row['VALUE'].max()]
            if not max_month_row.empty:
                max_month = max_month_row['Month'].values[0]
                color_grid[i, j] = np.array(mcolors.to_rgb(colors[max_month - 1]))
            else:
                color_grid[i, j] = np.array([1, 1, 1])  # White color for missing data

    # Store the color grid for the season
    seasonal_heatmaps[season] = color_grid

# Create a legend for months
legend_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
legend_colors = [colors[i] for i in range(12)]

# Plotting the heatmaps
fig, axs = plt.subplots(2, 2, figsize=(18, 14))
for ax, (season, color_grid) in zip(axs.flatten(), seasonal_heatmaps.items()):
    ax.imshow(color_grid)
    ax.set_title(f"Heatmap for {season}")
    
    # Replace y-tick labels with station names
    ax.set_yticks(np.arange(len(pivot_table.index)))
    ax.set_yticklabels([station_order[station] for station in pivot_table.index], fontsize=6)
    
    # Rotate x-tick labels
    ax.set_xticks(np.arange(len(pivot_table.columns)))
    ax.set_xticklabels(pivot_table.columns, rotation=90, fontsize=6)  # Adjust the rotation angle as needed
    
    ax.set_xlabel("Year")
    ax.set_ylabel("Station")

plt.tight_layout(rect=[0, 0.05, 1, 0.95])
fig.savefig("75gauge_3d_maxxxheatmappas.jpg", dpi=300)

plt.show()

In [ ]:
#seasonal precipitation mm percentages donut plot
import matplotlib.pyplot as plt

# Define colors for each season
colors = ['#17becf', '#fcb4ca', '#FFB347', '#828E84']

# Percentage values for the years 2002-2012 and 2013-2023
percentages_2002_2012 = [41.8, 21.7, 4.6, 31.9]
percentages_2013_2023 = [34.6, 21.9, 7.2, 36.3]

# Labels for the seasons
seasons = ['WINTER', 'SPRING', 'SUMMER', 'AUTUMN']

# Create a figure with two subplots (donut plots)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot for the years 2002-2012
wedges, texts, autotexts = ax1.pie(
    percentages_2002_2012,
    labels=None,
    colors=colors,
    startangle=90,
    pctdistance=0.85,
    wedgeprops={'width': 0.3, 'edgecolor': 'w'},  # Adjust the width for the donut
    autopct='%1.1f%%'
)

for i, p in enumerate(wedges):
    p.set(label=seasons[i])

for t in texts:
    t.set(size=12)

for at in autotexts:
    at.set(size=12, weight="bold", color="white", va='center')

ax1.axis('equal')  # Equal aspect ratio ensures that the donut is circular
ax1.set_title('2002-2012: 444 802 mm')

# Plot for the years 2013-2023
wedges, texts, autotexts = ax2.pie(
    percentages_2013_2023,
    labels=None,
    colors=colors,
    startangle=90,
    pctdistance=0.85,
    wedgeprops={'width': 0.3, 'edgecolor': 'w'},  # Adjust the width for the donut
    autopct='%1.1f%%'
)

for i, p in enumerate(wedges):
    p.set(label=seasons[i])

for t in texts:
    t.set(size=12)

for at in autotexts:
    at.set(size=12, weight="bold", color="white", va='center')

ax2.axis('equal')  # Equal aspect ratio ensures that the donut is circular
ax2.set_title('2013-2023: 408 769 mm')

# Add a unique legend
# Create a legend at the top
legend_elements = [Patch(color=color, label=season) for color, season in zip(colors, seasons)]
fig.legend(handles=legend_elements, loc="upper center", bbox_to_anchor=(0.5, 1.05), ncol=len(seasons))

plt.tight_layout()
fig.savefig("75gaugedonut_plot.jpg", dpi=300)
plt.show()


In [ ]:
# seasonal precipitation events percentage donut plot
import matplotlib.pyplot as plt

# Define colors for each season
colors = ['#17becf', '#fcb4ca', '#FFB347', '#828E84']

# Percentage values for the years 2002-2012 and 2013-2023
percentages_2002_2012 = [40.2, 22.4, 5.5, 31.9]
percentages_2013_2023 = [35.2, 24.6, 8.0, 32.2]

# Labels for the seasons
seasons = ['WINTER', 'SPRING', 'SUMMER', 'AUTUMN']

# Create a figure with two subplots (donut plots)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot for the years 2002-2012
wedges, texts, autotexts = ax1.pie(
    percentages_2002_2012,
    labels=None,
    colors=colors,
    startangle=90,
    pctdistance=0.85,
    wedgeprops={'width': 0.3, 'edgecolor': 'w'},  # Adjust the width for the donut
    autopct='%1.1f%%'
)

for i, p in enumerate(wedges):
    p.set(label=seasons[i])

for t in texts:
    t.set(size=12)

for at in autotexts:
    at.set(size=12, weight="bold", color="white", va='center')

ax1.axis('equal')  # Equal aspect ratio ensures that the donut is circular
ax1.set_title('2002-2012: 68 867 events')

# Plot for the years 2013-2023
wedges, texts, autotexts = ax2.pie(
    percentages_2013_2023,
    labels=None,
    colors=colors,
    startangle=90,
    pctdistance=0.85,
    wedgeprops={'width': 0.3, 'edgecolor': 'w'},  # Adjust the width for the donut
    autopct='%1.1f%%'
)

for i, p in enumerate(wedges):
    p.set(label=seasons[i])

for t in texts:
    t.set(size=12)

for at in autotexts:
    at.set(size=12, weight="bold", color="white", va='center')

ax2.axis('equal')  # Equal aspect ratio ensures that the donut is circular
ax2.set_title('2013-2023: 64 882 events')

# Add a unique legend
ax2.legend(wedges, seasons, title="Seasons", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

plt.tight_layout()
fig.savefig("75gaugedonut_plotevents.jpg", dpi=300)
plt.show()


In [ ]:
#simple ratio

import numpy as np
import matplotlib.pyplot as plt

# Define colors and year ranges
year_ranges = [(2002, 2012)]

# Define colors and width per category
colors = {
    'Volumes': 'blue',
    'Events': 'grey'
}
bar_width = 0.15

# Define the filters for the categories
filters = [
    (1, 2, 'Volumes'),
    (2.001, 6, 'Events')
]

# Define the y-values for the bars for each time interval (replace with your desired values)
data_per_interval = [
    {
        'Spring': [0.92, 1.02],  # Replace with your values for the first interval
        'Summer': [1.43, 1.36],  # Replace with your values for the first interval
        'Autumn': [1.04, 0.74],  # Replace with your values for the first interval
        'Winter': [0.76, 0.82],  # Replace with your values for the first interval
    }
]

# Define the labels for the seasons
season_labels = ['Spring', 'Summer', 'Autumn', 'Winter']

x = np.arange(len(season_labels))

for idx, (start_year, end_year) in enumerate(year_ranges):
    fig, ax = plt.subplots(figsize=(7, 2))
    current_data = data_per_interval[idx]

    for i, (category, color) in enumerate(zip(filters, colors.values())):
        category_name = category[2]
        y_values = [current_data[season][i] for season in season_labels]
        bars = ax.bar(x + i * bar_width, y_values, color=color, width=bar_width, label=f'{category_name}')
        
        # Add y-value annotations on top of each bar with a specified font size
        annotation_fontsize = 8  # Adjust the font size as needed
        for bar, y in zip(bars, y_values):
            ax.text(bar.get_x() + bar.get_width() / 2, y, f'{y:.2f}', ha='center', va='bottom', fontsize=annotation_fontsize)

    ax.set_xticks(x + (len(filters) - 1) * bar_width / 2)
    ax.set_xticklabels(season_labels)  # Label x-axis with season names
    ax.set_ylim(0, 2)  # Adjust the y-axis limit as needed
    ax.set_title(f'Simple ratio of volumes and events per season')
    
    # Set the size (linewidth) of the dashed horizontal line
    dashed_line_size = 0.5  # Adjust the line size as needed
    ax.axhline(y=1, color='red', linestyle='--', label='Y=1', linewidth=dashed_line_size)
    
    plt.tight_layout()
    fig.savefig("75gaugeSimple ratio of volumes and events per season.jpg", dpi=300)
    plt.show()

In [ ]:
#sicily map

import folium
import pandas as pd

# Read data from the COO CSV file
df = pd.read_csv("merged_COO.csv")

# Create a map centered around a location (you can choose any location)
m = folium.Map(location=[37.5, 14], zoom_start=8)

# Define a dictionary to map locations to marker colors
location_colors = {
    'CATANIA': 'red',
    'PALERMO': 'red',
    'TRAPANI FONTANASALSA': 'red',
    'AGRIGENTO SCIBICA': 'red',
    'CALTANISSETTA': 'red',
    'MESSINA': 'red',
    'ENNA': 'red',
    'RAGUSA': 'red',
    'SIRACUSA': 'red'
}

# Iterate through the DataFrame and add markers with different colors for specific locations
for index, row in df.iterrows():
    if not pd.isnull(row["EST"]) and not pd.isnull(row["NORD"]):
        location = row["LOCATION"]
        color = location_colors.get(location, 'black')  # Default to blue if location is not in the dictionary
        folium.CircleMarker(
            location=[float(row["NORD"]), float(row["EST"])],
            radius=3,  # Adjust the radius as needed
            popup=location,
            fill=True,
            fill_color=color,  # Use the color from the dictionary
            color=color,
            fill_opacity=0.6,
        ).add_to(m)

# Save the map to an HTML file
m


In [ ]:
# monthly dry and wet days trend plot

import csv
import matplotlib.pyplot as plt
import numpy as np

def plot_monthly_trends(csv_file):
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row

        month_data = {}  # Store monthly data for each month

        for row in reader:
            month = row[0]
            year = month[:4]  # Extract the year from the month
            wet_days = int(row[1])
            dry_days = int(row[2])

            month_number = int(month[5:])  # Extract the month number (e.g., 01, 02, etc.)
            month_label = month_number  # Initialize month label as the number

            if month_number == 1:
                month_label = 'January'
            # Rest of the month labels...

            if month_label in month_data:
                month_data[month_label]['years'].append(year)
                month_data[month_label]['wet_days'].append(wet_days)
                month_data[month_label]['dry_days'].append(dry_days)
            else:
                month_data[month_label] = {
                    'years': [year],
                    'wet_days': [wet_days],
                    'dry_days': [dry_days]
                }

    # Plotting the values for each month
    fig, axs = plt.subplots(6, 2, figsize=(15, 30))
    fig.subplots_adjust(hspace=0.5)

    for i, (month, data) in enumerate(month_data.items()):
        years = data['years']
        wet_days = data['wet_days']
        dry_days = data['dry_days']
        ax = axs[i // 2, i % 2]

        # Plot wet days
        ax.plot(years, wet_days, marker='o', label='Wet Days')
        ax.set_xlabel('Year')
        ax.set_ylabel('Wet Days')
        ax.set_title(f'{month} Wet Days')
        ax.grid(True)
        ax.set_xticks(years)
        ax.set_xticklabels(years, rotation='vertical')

        # Calculate and plot the trendline for wet days
        z_wet = np.polyfit(range(len(wet_days)), wet_days, 1)
        p_wet = np.poly1d(z_wet)
        ax.plot(years, p_wet(range(len(wet_days))), color='blue', label='Trend (Wet Days)')

        # Plot dry days
        ax.plot(years, dry_days, marker='o', label='Dry Days')
        ax.set_xlabel('Year')
        ax.set_ylabel('Dry Days')
        ax.set_title(f'{month} Dry Days')
        ax.grid(True)
        ax.set_xticks(years)
        ax.set_xticklabels(years, rotation='vertical')

        # Calculate and plot the trendline for dry days
        z_dry = np.polyfit(range(len(dry_days)), dry_days, 1)
        p_dry = np.poly1d(z_dry)
        ax.plot(years, p_dry(range(len(dry_days))), color='red', label='Trend (Dry Days)')

        # Show slope values on the plot
        slope_wet = np.round(z_wet[-2], 2)
        slope_dry = np.round(z_dry[0], 2)
        ax.text(0.02, 0.92, f'Slope (Wet): {slope_wet}', transform=ax.transAxes, fontsize=10, color='blue')
        ax.text(0.02, 0.86, f'Slope (Dry): {slope_dry}', transform=ax.transAxes, fontsize=10, color='red')

        #ax.legend()

    plt.tight_layout()
    plt.savefig('monthlywetdrytrend.jpg', format='jpg', quality=350)
    plt.show()

# CSV file path
csv_file = 'tpmonthlywetnddry.csv'

# Plot the data for each month with a trendline and save as a single JPG image
plot_monthly_trends(csv_file)

In [ ]:
# monthly dry and wet days trend plot superimposed

import glob

# Find all the CSV files in the output directory
csv_files = glob.glob(os.path.join(output_dir, '*_annual_data.csv'))

# Loop over the CSV files and create a plot for each one
for csv_file in csv_files:
    # Read the CSV file into a pandas DataFrame
    df_annual = pd.read_csv(csv_file)

    # Create a figure and an axis
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot count_0 on the first y-axis
    ax1.plot(df_annual['year'], df_annual['count_0'], marker='o', color='r', label='n.o of dry days')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('n.o of dry days', color='r')
    ax1.tick_params(axis='y', labelcolor='r')
    ax1.set_ylim([100, 365])  # set y-axis limits for count_0

    # Create a second y-axis and plot sum on it
    ax2 = ax1.twinx()
    ax2.plot(df_annual['year'], df_annual['sum'], marker='o', color='b', label='annual total mm of precipitation')
    ax2.set_ylabel('annual total mm of precipitation', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    ax2.set_ylim([100, 900])  


    # Create a third y-axis and plot q on it
    ax3 = ax1.twinx()
    ax3.spines['right'].set_position(('axes', 1.15))
    ax3.plot(df_annual['year'], df_annual['q'], marker='o', color='g', label='q')
    ax3.set_ylabel('q', color='g')
    ax3.tick_params(axis='y', labelcolor='g')
    ax3.set_ylim([1.2, 2])  

    # Set the title for the figure
    interval_name = os.path.splitext(os.path.basename(csv_file))[0].replace('_annual_q', '')
    plt.title(f'{interval_name} - n.o of dry days, total mm anual precipitation, and q')
    
    # Adjust padding/margins to make room for third axis
    plt.subplots_adjust(right=0.80)

    # Save the figure as an image
    #plt.savefig(os.path.join(output_dir, f'{interval_name}_superimposed_plot.jpg'), dpi=350)
    # Save the figure as an image with higher DPI and tight bounding box
    plt.savefig(os.path.join(output_dir, f'{interval_name}_superimposed_plot.jpg'), dpi=300, bbox_inches='tight')


    # Show the figure
    plt.show()


In [ ]:
#superimposed dry, mm and q exponent

import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

output_dir = 'output'
# Find all the CSV files in the output directory
csv_files = glob.glob(os.path.join(output_dir, '*_annual_data.csv'))

# Loop over the CSV files and create a plot for each one
for csv_file in csv_files:
    # Read the CSV file into a pandas DataFrame
    df_annual = pd.read_csv(csv_file)

    # Create a figure and an axis
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot count_0 on the first y-axis
    ax1.plot(df_annual.index, df_annual['count_0'], marker='o', color='r', label='n.o of dry days')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('n.o of dry days', color='r')
    ax1.tick_params(axis='y', labelcolor='r')
    ax1.set_ylim([0, 400])  # set y-axis limits for count_0

    # Create a second y-axis and plot sum on it
    ax2 = ax1.twinx()
    ax2.plot(df_annual.index, df_annual['sum'], marker='o', color='b', label='annual total mm of precipitation')
    ax2.set_ylabel('annual total mm of precipitation', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    ax2.set_ylim([0, 1800])  


    # Create a third y-axis and plot q on it
    ax3 = ax1.twinx()
    ax3.spines['right'].set_position(('axes', 1.15))
    ax3.plot(df_annual.index, df_annual['q'], marker='o', color='g', label='q')
    ax3.set_ylabel('q', color='g')
    ax3.tick_params(axis='y', labelcolor='g')
    ax3.set_ylim([0, 3])  

    # Set the title for the figure
    interval_name = os.path.splitext(os.path.basename(csv_file))[0].replace('_annual_q', '')
    plt.title(f'{interval_name} - n.o of dry days, total mm annual precipitation, and q')
    
    # Adjust padding/margins to make room for third axis
    plt.subplots_adjust(right=0.80)

    # Save the figure as an image
    #plt.savefig(os.path.join(output_dir, f'{interval_name}_superimposed_plot.jpg'), dpi=350)
    # Save the figure as an image with higher DPI and tight bounding box
    plt.savefig(os.path.join(output_dir, f'{interval_name}_superimposed_plot.jpg'), dpi=300, bbox_inches='tight')


    # Show the figure
    plt.show()

In [ ]:
# Sicily map dry days

import pandas as pd
import folium
from IPython.display import display
from branca.element import MacroElement

# Merge pivot table and coordinates data
#piv = pd.pivot_table(df, index=['Year'], columns=['LOCATION'], values=df.columns[3])
piv = pd.pivot_table(
    df,
    index=['Year'],
    columns=['LOCATION'],
    values='Count',
    aggfunc='sum'  # You can use 'sum', 'mean', etc., as appropriate.
)

coordinates = pd.read_csv('COO.csv')
merged_data = pd.merge(piv.stack().reset_index(), coordinates, on='LOCATION')

# Create folium map object
for year in merged_data['Year'].unique():
    year_data = merged_data[merged_data['Year'] == year]
    m = folium.Map(location=[37.5, 14], zoom_start=7.5)

    # Define the color scheme and opacity for the markers
    color_scale = folium.StepColormap(['white', 'yellow', 'gold', '#8b0000','black'], index=[170,195, 225,255,285])

    # Add markers to map for each location with radius proportional to value
    for index, row in year_data.iterrows():
        tooltip = row['LOCATION']
        popup = f"Year: {row['Year']}, Value: {row[0]}"
        radius = 7  # Set a fixed radius for all circles
        color = color_scale(row[0])
        folium.CircleMarker(location=[row['NORD'], row['EST']], radius=radius, tooltip=tooltip, popup=popup,
                            fill_color=color, color=color, fill_opacity=0.8, weight=1).add_to(m)
        
    # Add title
    title_html = f"""
        <h3 style="position: fixed;
                   top: 50px; left: 50px; width: 200px;
                   z-index:9999; font-size:16px; font-weight:bold; padding: 5px;">
        Dry days for year {year}
        </h3>
        """
    m.get_root().html.add_child(folium.Element(title_html))

    # Add color legend to map
    #color_scale.caption = 'Count'
    #m.add_child(color_scale)

    # Add custom HTML legend to map
    legend_html = """
        <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 200px;
                    z-index:9999; font-size:14px; font-weight:bold; padding: 5px;
                    background-color: white; opacity: 0.9;
                    border: 2px solid grey;">
            <ul style="list-style-type:none; padding-left:0;">
                <li><span style="background-color:white; border:1px solid grey; padding:2px 8px;"></span>  dd &le; 195 Temperate
                <li><span style="background-color:yellow; border:1px solid grey; padding:2px 8px;"></span> dd&le; 225 soft drought
                <li><span style="background-color:gold; border:1px solid grey; padding:2px 8px;"></span> dd&le; 255 drought
                <li><span style="background-color:#8b0000; border:1px solid grey; padding:2px 8px;"></span> dd&le; 285 severe drought
                <li><span style="background-color:black; border:1px solid grey; padding:2px 8px;"></span> dd&ge;285 subtropical drought
     </div>
        """
    m.get_root().html.add_child(folium.Element(legend_html))
    # Save map to HTML file
    #m.save(f'dry_days_{year}.html')

    
  

    
    # Display map for current year
    display(m)

In [ ]:
# sicily map wet days

import pandas as pd
import folium
from IPython.display import display
from branca.element import MacroElement
from selenium import webdriver
import time

# Merge pivot table and coordinates data
piv = pd.pivot_table(df, index=['Year'], columns=['LOCATION'], values=df.columns[2])
coordinates = pd.read_csv('COO.csv')
merged_data = pd.merge(piv.stack().reset_index(), coordinates, on='LOCATION')

# Create folium map object
for year in merged_data['Year'].unique():
    year_data = merged_data[merged_data['Year'] == year]
    m = folium.Map(location=[37.5, 14], zoom_start=7)

    # Define the color scheme and opacity for the markers
    color_scale = folium.StepColormap(['#D2B48C', '#87CEFA','#6495ED','#FFCCFF', '#FF99FF','magenta','#9900FF'], index=[50,300, 600, 900,1200,1400,1600])

    # Add markers to map for each location with radius proportional to value
    for index, row in year_data.iterrows():
        tooltip = row['LOCATION']
        popup = f"Year: {row['Year']}, Value: {row[0]}"
        value = row[0]
        if value > 0: # Check if value is greater than 0
            radius = 7  # Set a fixed radius for all circles
            color = color_scale(value)
            folium.CircleMarker(location=[row['NORD'], row['EST']], radius=radius, tooltip=tooltip, popup=popup,
                                fill_color=color, color=color, fill_opacity=0.8, weight=1).add_to(m)

    # Save map to HTML file
    temp_html = f'temp_map_year_{year}.html'
    m.save(temp_html)

    # Use selenium to capture map as PNG image
    browser = webdriver.Chrome()
    browser.get(f'file://{temp_html}')
    time.sleep(5)  # Wait for map to load
    browser.save_screenshot(f'prec_year_{year}.png')
    browser.quit()

    # Display map for current year
    display(m)
